# ADS 509 Sentiment Assignment

This notebook holds the Sentiment Assignment for Module 6 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In a previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we apply sentiment analysis to those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [2]:
# Add any additional import statements you need here
import nltk

In [3]:
# change `data_location` to the location of the folder on your machine.
data_location = "D:/USD-ADS_graduate/ADS-509/Module 1/ADS_509_module1/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
tidy_text_file = "tidytext_sentiments.txt"

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A Pandas data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [4]:
# Read in the lyrics data

"""
read the files into lists and use the lists to make a dataframe
"""
lyrics_path = data_location + lyrics_folder
artists = os.listdir(lyrics_path)

a = []
b = []
c = []

for artist in artists:
    raw_song = os.listdir(lyrics_path+artist)
    for song in raw_song:
        a.append(str(artist))
        f = open(lyrics_path+artist+'/'+song, "r")
        song = re.search('_(.+?).txt',song).group(1)
        b.append(song)  
        c.append(f.read())

lyrics_df = pd.DataFrame({'artist':a,
                           'song':b,
                           'song_content':c})

lyrics_df.sample(10)

,artist,song,song_content
35,Nicki Minaj,freestyle,"['Freestyle']\n\nI don't want to sound mad, th..."
27,Nicki Minaj,40bars,"['40 Bars']\n\nHa ha, ha ha\nIt's your girl Ni..."
20,Mariah Carey,tilltheendoftime,['Till The End Of Time']\n\nLove you till the ...
0,Mariah Carey,allinyourmind,['All In Your Mind']\n\nCome closer\nYou seem ...
13,Mariah Carey,nowthatiknow,['Now That I know']\n\nNow that I know that yo...
21,Mariah Carey,tobearoundyou,['To Be Around You']\n\nSomething that you do\...
22,Mariah Carey,vanishing,['Vanishing']\n\nIf I could recapture all of t...
1,Mariah Carey,aloneinlove,['Alone In Love']\n\nSwept me away\nBut now I'...
14,Mariah Carey,prisoner,['Prisoner']\n\nYou don't think that I'll be s...
51,Nicki Minaj,wuchooknow,"['Wuchoo Know']\n\nChin checker, chin checker\..."


In [5]:
# Read in the twitter data

"""
manually read those files into dataframe's columns
"""
twitter_path = data_location + twitter_folder
twitter_files = os.listdir(twitter_path)

mariah_df = pd.DataFrame()
nicki_df = pd.DataFrame()

mariah_des = pd.read_csv(os.path.join(twitter_path, 'MariahCarey_followers.txt'),sep='\t')['description']
nicki_des = pd.read_csv(os.path.join(twitter_path, 'NICKIMINAJ_followers.txt'),sep='\t')['description']

mariah_df['artist'] = ['Mariah Carey']*len(mariah_des)
mariah_df['description'] = mariah_des

nicki_df['artist'] = ['Nicki Minaj']*len(nicki_des)
nicki_df['description'] = nicki_des

twitter_df = pd.concat([mariah_df,nicki_df])
twitter_df['description'] = twitter_df['description'].astype(str)

twitter_df.sample(10)

,artist,description
38654,Mariah Carey,unless you are willing to
7406,Nicki Minaj,when i was going to highschool i threw a kid i...
7401,Mariah Carey,nan
43634,Nicki Minaj,nan
79588,Nicki Minaj,nan
64537,Nicki Minaj,nan
28100,Nicki Minaj,nan
95595,Mariah Carey,Denizci
59807,Nicki Minaj,nan
48934,Mariah Carey,I’m here. Hello 🤚


In [6]:
# Read in the positive and negative words and the
# tidytext sentiment. Store these so that the positive
# words are associated with a score of +1 and negative words
# are associated with a score of -1. You can use a dataframe or a 
# dictionary for this.
good_words = pd.read_csv('D:/USD-ADS_graduate/ADS-509/Module 6/ADS_509_module6/positive-words.txt',sep='\t',encoding = 'latin-1')
bad_words = pd.read_csv('D:/USD-ADS_graduate/ADS-509/Module 6/ADS_509_module6/negative-words.txt',sep='\t',encoding = 'latin-1')
tidytext = pd.read_csv('D:/USD-ADS_graduate/ADS-509/Module 6/ADS_509_module6/tidytext_sentiments.txt',sep='\t',encoding = 'latin-1')

In [7]:
tidytext = tidytext.drop(['lexicon'],axis=1)
good_words = good_words.drop(range(33))
bad_words = bad_words.drop(range(33))

In [8]:
# append the good words and bad words to the tidytext dataframe 
good_words['sentiment'] = 'positive'
good_words.rename(columns={good_words.columns[0]: "word"}, inplace = True)
bad_words['sentiment'] = 'negative'
bad_words.rename(columns={bad_words.columns[0]: "word"}, inplace = True)

In [9]:
# concatenate all three dfs and remove duplicates and reset the index
word_df = pd.concat([tidytext, good_words, bad_words]).drop_duplicates(subset=['word']).reset_index(drop=True)
word_df

,word,sentiment
0,abandon,negative
1,abandoned,negative
2,abandonment,negative
3,abba,positive
4,abduction,negative
...,...,...
11403,tremendous,positive
11404,upturn,positive
11405,upturns,positive
11406,vibrancy,positive


In [10]:
# check for duplicated words
word_df.loc[word_df['word'].duplicated()].sum()

word         0.0
sentiment    0.0
dtype: float64

In [11]:
# map the positive words are associated with a score 
# of +1 and negative words are associated with a score of -1
sentiment_map = {'positive':1, 'negative':-1}
word_df["sentiment"] = word_df["sentiment"].map(sentiment_map)
word_df["sentiment"] = word_df['sentiment'].astype(int) # make sure they are integers
word_df

,word,sentiment
0,abandon,-1
1,abandoned,-1
2,abandonment,-1
3,abba,1
4,abduction,-1
...,...,...
11403,tremendous,1
11404,upturn,1
11405,upturns,1
11406,vibrancy,1


## Sentiment Analysis on Songs

In this section, score the sentiment for all the songs for both artists in your data set. Score the sentiment by manually calculating the sentiment using the combined lexicons provided in this repository. 

After you have calculated these sentiments, answer the questions at the end of this section.


In [16]:
# your code here


5

### Questions

Q: Overall, which artist has the higher average sentiment per song? 

A: <!-- Your answer here -->

---

Q: For your first artist, what songs have the highest and lowest sentiments? Print those songs to the screen.

A: <!-- Your answer here -->

---

Q: For your second artist, what songs have the highest and lowest sentiments? Print those songs to the screen.

A: <!-- Your answer here -->

---

Q: Plot the distributions of the sentiment scores for both artists. You can use `seaborn` to plot densities or plot histograms in matplotlib.




## Sentiment Analysis on Twitter Descriptions

In this section, define two sets of emojis you designate as positive and negative. Make sure to have at least 10 emojis per set. You can learn about the most popular emojis on Twitter at [the emojitracker](https://emojitracker.com/). 

Associate your positive emojis with a score of +1, negative with -1. Score the average sentiment of your two artists based on the Twitter descriptions of their followers. The average sentiment can just be the total score divided by number of followers. 

In [13]:
# your code here

Q: What is the average sentiment of your two artists? 

A: <!-- Your answer here --> 

---

Q: Which positive emoji is the most popular for each artist? Which negative emoji? 

A: <!-- Your answer here --> 

